In [7]:
import csv
import zipfile
import random

!wget --no-check-certificate "https://github.com/andynosbig/moviele/archive/refs/heads/main.zip" -O "/tmp/main.zip"
zip_ = zipfile.ZipFile('/tmp/main.zip', 'r')
zip_.extractall('/tmp')
zip_.close()

filmsByTitle = {}
with open("/tmp/moviele-main/combined.tsv", 'r') as basics:
    for row in csv.DictReader(basics, delimiter='\t'):
        filmsByTitle[row['title']] = row
        filmsByTitle[row['title']].pop('id')
        
        filmsByTitle[row['title']]['genres'] = row['genres'].split(',')
        filmsByTitle[row['title']]['principals'] = row['principals'].split(',')
        filmsByTitle[row['title']].pop('title')

principalNames = {}
with open("/tmp/moviele-main/principals.tsv", 'r') as basics:
    for row in csv.DictReader(basics, delimiter='\t'):
        principalNames[row['id']] = row['name']

--2022-02-19 21:37:27--  https://github.com/andynosbig/moviele/archive/refs/heads/main.zip
Resolving github.com (github.com)... 81.99.162.48, 2a02:8801:200:1::1
Connecting to github.com (github.com)|81.99.162.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/andynosbig/moviele/zip/refs/heads/main [following]
--2022-02-19 21:37:28--  https://codeload.github.com/andynosbig/moviele/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/main.zip’

/tmp/main.zip           [  <=>               ] 778.63K  2.37MB/s    in 0.3s    

2022-02-19 21:37:28 (2.37 MB/s) - ‘/tmp/main.zip’ saved [797315]



In [26]:
def get_random_item(list_):
    return list(list_)[random.randint(0, len(list_))]

def calc_list_similarity(listA, listB):
    return 100 * (len(set(listA)&set(listB)) / len(set(listA)|set(listB))), "'".join(list(set(listA)&set(listB)))
    
def test_guess(x):
    global current_results
    global random_film
    
    if x not in filmsByTitle:
        return

    if filmsByTitle[x] == random_film:
        return "Success!"

    guess_film = filmsByTitle[x]
    genre_percentage, genre_overlap = calc_list_similarity(guess_film['genres'], random_film['genres'])
    principals_percentage, principals_overlap = calc_list_similarity(guess_film['principal_ids'], random_film['principal_ids'])
    principal_name_overlap = [principalNames[x] for x in principals_overlap]
    year_diff = int(guess_film['startYear']) - int(random_film['startYear'])
    result = f"{x} - \nprincipal matches={principals_percentage}% - {principal_name_overlap}\ngenre matches={genre_percentage}% - {genre_overlap} - year difference={year_diff}"
    current_results += result + "\n\n"
    return current_results

def get_new_target():
    global current_results
    global random_film
    
    random_film = filmsByTitle["Harry Potter and the Sorcerer's Stone"]#get_random_item(filmsByTitle.values())
    print(f"hint,  {get_random_item(random_film['principals'])} is involved in this film")
    current_results = ""
    
def show_answer():
    print(random_film['title'])

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets

def run():
    output = widgets.Output()
    new_target_button = widgets.Button(description="Reset Target Film")
    display(new_target_button, output)
    new_target_button.on_click(get_new_target)

    get_answer_button = widgets.Button(description="Show Answer")
    display(get_answer_button, output)
    get_answer_button.on_click(show_answer)

    films = ["-"]
    films.extend(sorted(filmsByTitle.keys()))
    interact(test_guess, x=films)
    interact

run()

interactive(children=(Dropdown(description='x', options=('-', '12 Monkeys', '12 Years a Slave', '13 Assassins'…

<function __main__.test_film(x)>